In [1]:
import numpy as np #importing numpy
import pandas as pd #importing pandas
import PIL #importing pillow for image processing
from PIL import Image#importing image from pillow library
import skimage#importing skimage library
import os#importing os library
import time#importing time library

#Capture start time of the program
start_time = time.time()

In [2]:
from string import digits
#code to remove numbers in a string and assigning it to the variable
r_digits = str.maketrans('', '', digits)
#taking source data folder path to a varible
rawdata_path = 'D://DKIT//dissertation//Dataset_of_Tomato_Leaves//plantvillage//Preprocessed_data//32//data//'
#changing current librbay to source data
os.chdir(rawdata_path)
#taking sub-directories to a list
dirs = os.listdir()
directories = []
diseases = []
#Taking directories into a list and extract disease name from folder name
for x in dirs:
    res = x.translate(r_digits).lower()
    diseases.append(res)
    temp = rawdata_path + x + '//'
    directories.append(temp)

In [3]:
for i in directories:
    os.chdir(i)
    pics = os.listdir()
    #reading pics one by one in each directory
    for j in pics:
        image = Image.open(j,mode='r')
        size = (32,32)
        image = image.resize(size)
        image = image.save(j)

In [4]:
#creating different dataframes for saving data in different vector forms
#gray data
gray_df = pd.DataFrame(columns=['mean','disease'])
#rgb data
rgb_df = pd.DataFrame(columns=['mean','disease'])
#red data
red_df = pd.DataFrame(columns=['mean','disease'])
#green data
green_df = pd.DataFrame(columns=['mean','disease'])
#blue data
blue_df = pd.DataFrame(columns=['mean','disease'])

In [5]:
#extracting data from images
x = 0
#creating enpty directories to save the vector from in to a list
gray_test = []
rgb_final = []
red_final = []
green_final = []
blue_final = []
disease = []
#reading data directory by directory
for i in directories:
    os.chdir(i)
    k = diseases[x]
    pics = os.listdir()
    #reading pics one by one in each directory
    for j in pics:
        temp_list = []
        rgb_temp = []
        red_temp = []
        green_temp = []
        blue_temp = []
        gray = []
        data = []
        red = []
        green = []
        blue = []
        #opening the image in rgb format
        image = Image.open(j,mode='r')
        data = image.convert('RGB')
        data = np.array(data)
        #extracting only red components
        red = data[:,:,0]
        #extracting only green components
        green = data[:,:,1]
        #extracting only blue components
        blue = data[:,:,2]
        data = data.ravel()
        #saving rgb vector data into list
        rgb_final.append(data)
        rgb_temp.append(np.mean(data))
        rgb_temp.append(k)
        rgb_df.loc[len(rgb_df)] = rgb_temp
        #saving red channel data into dataframe and vector data to a list.
        red = red.ravel()
        red_final.append(red)
        red_temp.append(np.mean(red))
        red_temp.append(k)
        red_df.loc[len(red_df)] = red_temp
        #saving blue channel data into dataframe and vector data to a list.
        blue = blue.ravel()
        blue_final.append(blue)
        blue_temp.append(np.mean(blue))
        blue_temp.append(k)
        blue_df.loc[len(blue_df)] = blue_temp
        #saving green channel data into dataframe and vector data to a list.
        green = green.ravel()
        green_final.append(green)
        green_temp.append(np.mean(green))
        green_temp.append(k)
        green_df.loc[len(green_df)] = green_temp
        #opening the image in gray format
        gray = image.convert('L')
        gray = np.array(gray)
        gray = gray.ravel()
        gray_test.append(gray)
        temp_list.append(np.mean(gray))
        temp_list.append(k)       
        gray_df.loc[len(gray_df)] = temp_list
    x = x+1

In [6]:
#changing path
os.chdir('D://DKIT//dissertation//Dataset_of_Tomato_Leaves//plantvillage//Preprocessed_data//32//vector//')

In [7]:
#saving the lists as numpy arrays in the local system
np.save('gray_test',gray_test)
np.save('rgb_data',rgb_final)
np.save('red_data',red_final)
np.save('green_data',green_final)
np.save('blue_data',blue_final)

In [8]:
#saving the dataframe as csv files in the local system
gray_df.to_csv('gray_data.csv',index=False)
rgb_df.to_csv('rgb_data.csv',index=False)
red_df.to_csv('red_data.csv',index=False)
green_df.to_csv('green_data.csv',index=False)
blue_df.to_csv('blue_data.csv',index=False)

# KNN on rgb data

In [9]:
#reading numpy array data
rgb_test = np.load('rgb_data.npy')
#reading csv file
rgb_df = pd.read_csv('rgb_data.csv')

In [12]:
loaded = []
for i in rgb_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)

In [13]:
#assigning features and target
features = loaded
target = rgb_df['disease']
#splitting the data to 80 percent and 20 percent
from sklearn.model_selection import train_test_split
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=24)
#scaling the data using standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)

In [14]:
#K-Model on the data
from sklearn.neighbors import KNeighborsClassifier
kmodel = KNeighborsClassifier(n_neighbors=75)

In [15]:
#k-model fitting and accuracy
kmodel.fit(Ftrain,Ltrain)
print(kmodel.score(Ftest,Ltest))

0.5913312693498453


In [16]:
#importing classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

In [17]:
#printing results of the kmodel
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.36      0.84      0.51       345
                 early_blight       0.00      0.00      0.00       153
                      healthy       0.50      0.81      0.62       234
                  late_blight       0.59      0.40      0.48       309
                    leaf_mold       0.73      0.52      0.61       145
           septoria_leaf_spot       0.70      0.40      0.51       293
                  target_spot       0.58      0.29      0.38       240
          tomato_mosaic_virus       0.83      0.19      0.30        54
tomato_yellow_leaf_curl_virus       0.84      0.80      0.82       854
      two-spotted_spider_mite       0.52      0.56      0.54       280

                     accuracy                           0.59      2907
                    macro avg       0.57      0.48      0.48      2907
                 weighted avg       0.61      0.59      0.57      2907

[[2

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Logistic on rgb data

In [18]:
#importing logistic data
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='lbfgs', max_iter=10000)
model.fit(Ftrain,Ltrain)
print(model.score(Ftest,Ltest))

0.6838665290677675


In [19]:
#classification report
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.65      0.75      0.70       345
                 early_blight       0.31      0.32      0.32       153
                      healthy       0.67      0.65      0.66       234
                  late_blight       0.54      0.52      0.53       309
                    leaf_mold       0.58      0.57      0.57       145
           septoria_leaf_spot       0.67      0.64      0.66       293
                  target_spot       0.58      0.57      0.57       240
          tomato_mosaic_virus       0.62      0.74      0.68        54
tomato_yellow_leaf_curl_virus       0.90      0.88      0.89       854
      two-spotted_spider_mite       0.62      0.61      0.61       280

                     accuracy                           0.68      2907
                    macro avg       0.62      0.63      0.62      2907
                 weighted avg       0.69      0.68      0.68      2907

[[2

# SVM(linear) on RGB data

In [20]:
#importing svm algorithm
from sklearn.svm import SVC
#importing accuracy score
from sklearn.metrics import accuracy_score
#building svm model
clf = SVC(kernel='linear')
#fitting the model
clf.fit(Ftrain,Ltrain)
#assigning the predictions to y_pred
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.7337461300309598


In [21]:
#classification report
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.66      0.82      0.73       345
                 early_blight       0.37      0.37      0.37       153
                      healthy       0.76      0.81      0.79       234
                  late_blight       0.65      0.65      0.65       309
                    leaf_mold       0.66      0.61      0.64       145
           septoria_leaf_spot       0.71      0.66      0.68       293
                  target_spot       0.67      0.61      0.64       240
          tomato_mosaic_virus       0.77      0.85      0.81        54
tomato_yellow_leaf_curl_virus       0.91      0.87      0.89       854
      two-spotted_spider_mite       0.70      0.67      0.68       280

                     accuracy                           0.73      2907
                    macro avg       0.69      0.69      0.69      2907
                 weighted avg       0.74      0.73      0.73      2907

[[2

# SVM(poly) on RGB data

In [22]:
#building poly model
svm_poly = SVC(kernel='poly')
#fitting the model
svm_poly.fit(Ftrain,Ltrain)
y_pred = svm_poly.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.7447540419676643


In [23]:
#classification report
y_predict = svm_poly.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.62      0.86      0.72       345
                 early_blight       0.64      0.14      0.23       153
                      healthy       0.81      0.90      0.85       234
                  late_blight       0.87      0.61      0.72       309
                    leaf_mold       0.80      0.60      0.69       145
           septoria_leaf_spot       0.73      0.66      0.69       293
                  target_spot       0.68      0.54      0.60       240
          tomato_mosaic_virus       0.53      0.17      0.25        54
tomato_yellow_leaf_curl_virus       0.77      0.96      0.86       854
      two-spotted_spider_mite       0.75      0.75      0.75       280

                     accuracy                           0.74      2907
                    macro avg       0.72      0.62      0.64      2907
                 weighted avg       0.74      0.74      0.72      2907

[[2

# SVM(RBF) on RGB data

In [24]:
#building the svm(rbf) model
svm_rbf = SVC(kernel='rbf')
#fitting the model
svm_rbf.fit(Ftrain,Ltrain)
y_pred = svm_rbf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.8469212246302029


In [25]:
#classification report
y_predict = svm_rbf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.87      0.91      0.89       345
                 early_blight       0.63      0.42      0.50       153
                      healthy       0.96      0.95      0.95       234
                  late_blight       0.79      0.82      0.81       309
                    leaf_mold       0.81      0.69      0.74       145
           septoria_leaf_spot       0.79      0.78      0.79       293
                  target_spot       0.76      0.78      0.77       240
          tomato_mosaic_virus       0.77      0.76      0.77        54
tomato_yellow_leaf_curl_virus       0.92      0.96      0.94       854
      two-spotted_spider_mite       0.81      0.82      0.82       280

                     accuracy                           0.85      2907
                    macro avg       0.81      0.79      0.80      2907
                 weighted avg       0.84      0.85      0.84      2907

[[3

# SVM(sigmoid) on RGB data

In [26]:
#building the sigmoid model
svm_si = SVC(kernel='sigmoid')
#fitting the model
svm_si.fit(Ftrain,Ltrain)
y_pred = svm_si.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.5455796353629171


In [27]:
#classification report
y_predict = svm_si.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.58      0.62      0.60       345
                 early_blight       0.21      0.08      0.12       153
                      healthy       0.72      0.63      0.67       234
                  late_blight       0.37      0.31      0.33       309
                    leaf_mold       0.65      0.23      0.34       145
           septoria_leaf_spot       0.25      0.57      0.34       293
                  target_spot       0.51      0.38      0.44       240
          tomato_mosaic_virus       0.68      0.24      0.36        54
tomato_yellow_leaf_curl_virus       0.80      0.82      0.81       854
      two-spotted_spider_mite       0.53      0.40      0.46       280

                     accuracy                           0.55      2907
                    macro avg       0.53      0.43      0.45      2907
                 weighted avg       0.58      0.55      0.54      2907

[[2

# Random Forest on RGB 

In [28]:
#importing the random forest
from sklearn.ensemble import RandomForestClassifier

In [29]:
#building the model
classifier_rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=5,n_estimators=100, oob_score=True)

In [30]:
#fitting the model
classifier_rf.fit(Ftrain, Ltrain)

RandomForestClassifier(max_depth=5, n_jobs=-1, oob_score=True, random_state=42)

In [31]:
classifier_rf.oob_score_

0.5579834824501032

In [32]:
#predicting the model
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
#printing the classification report
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.74      0.58      0.65       345
                 early_blight       0.00      0.00      0.00       153
                      healthy       0.56      0.91      0.69       234
                  late_blight       0.68      0.27      0.39       309
                    leaf_mold       0.78      0.34      0.47       145
           septoria_leaf_spot       0.57      0.44      0.50       293
                  target_spot       0.54      0.40      0.46       240
          tomato_mosaic_virus       0.00      0.00      0.00        54
tomato_yellow_leaf_curl_virus       0.52      0.98      0.68       854
      two-spotted_spider_mite       0.77      0.17      0.28       280

                     accuracy                           0.57      2907
                    macro avg       0.52      0.41      0.41      2907
                 weighted avg       0.57      0.57      0.51      2907

[[2

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Gaussian Naive Bayes on RGB data

In [33]:
#importing the gaussian bayes
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
#fitting the model
gnb.fit(Ftrain, Ltrain)

GaussianNB()

In [34]:
#predicting the output
y_pred = gnb.predict(Ftest)

In [35]:
#importing the metrics
from sklearn import metrics
metrics.accuracy_score(Ltest, y_pred)

0.5063639490884073

In [36]:
#printing the classification report
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.53      0.76      0.62       345
                 early_blight       0.23      0.31      0.26       153
                      healthy       0.42      0.71      0.53       234
                  late_blight       0.51      0.28      0.36       309
                    leaf_mold       0.53      0.44      0.48       145
           septoria_leaf_spot       0.44      0.46      0.45       293
                  target_spot       0.54      0.45      0.49       240
          tomato_mosaic_virus       0.19      0.72      0.31        54
tomato_yellow_leaf_curl_virus       0.89      0.51      0.65       854
      two-spotted_spider_mite       0.40      0.47      0.44       280

                     accuracy                           0.51      2907
                    macro avg       0.47      0.51      0.46      2907
                 weighted avg       0.58      0.51      0.52      2907

[[2

# Bernouli Naive bayes on RGB data

In [37]:
#importing bernouli naive bayes
from sklearn.naive_bayes import BernoulliNB

In [38]:
#fitting the model
bnb = BernoulliNB()
bnb.fit(Ftrain, Ltrain)

BernoulliNB()

In [39]:
#predicting the output
y_pred = bnb.predict(Ftest)

In [40]:
metrics.accuracy_score(Ltest, y_pred)

0.43034055727554177

In [41]:
#classification report
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.42      0.71      0.53       345
                 early_blight       0.14      0.19      0.16       153
                      healthy       0.43      0.60      0.50       234
                  late_blight       0.15      0.10      0.12       309
                    leaf_mold       0.59      0.40      0.48       145
           septoria_leaf_spot       0.38      0.57      0.45       293
                  target_spot       0.51      0.46      0.48       240
          tomato_mosaic_virus       0.19      0.52      0.28        54
tomato_yellow_leaf_curl_virus       0.87      0.35      0.50       854
      two-spotted_spider_mite       0.44      0.50      0.46       280

                     accuracy                           0.43      2907
                    macro avg       0.41      0.44      0.40      2907
                 weighted avg       0.52      0.43      0.43      2907

[[2

# Decision Tree on RGB data

In [47]:
#importing the decision tree
from sklearn.tree import DecisionTreeClassifier
classifier_tree = DecisionTreeClassifier()

In [48]:
#fitting the model
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.5042999656002752

In [49]:
#classification report
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.49      0.57      0.53       345
                 early_blight       0.14      0.14      0.14       153
                      healthy       0.60      0.67      0.63       234
                  late_blight       0.46      0.46      0.46       309
                    leaf_mold       0.37      0.36      0.36       145
           septoria_leaf_spot       0.43      0.41      0.42       293
                  target_spot       0.34      0.31      0.32       240
          tomato_mosaic_virus       0.10      0.11      0.10        54
tomato_yellow_leaf_curl_virus       0.74      0.69      0.72       854
      two-spotted_spider_mite       0.37      0.38      0.38       280

                     accuracy                           0.50      2907
                    macro avg       0.40      0.41      0.41      2907
                 weighted avg       0.51      0.50      0.51      2907

[[1

# KNN on grayscale data

In [50]:
#reading numpy array data
gray_test = np.load('gray_test.npy')
#reading csv file
gray_df = pd.read_csv('gray_data.csv')

In [51]:
#reading each array
loaded = []
for i in gray_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)

In [52]:
#assigning features and target
features = loaded
target = gray_df['disease']
#splitting the dataset to train and test
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
#scaling the data
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)

In [53]:
#fitting the model
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.4864121087031304

In [54]:
#classification model
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.38      0.77      0.50       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.38      0.60      0.46       258
                  late_blight       0.37      0.29      0.32       316
                    leaf_mold       0.74      0.46      0.56       145
           septoria_leaf_spot       0.56      0.39      0.46       262
                  target_spot       0.70      0.17      0.27       229
          tomato_mosaic_virus       1.00      0.03      0.07        58
tomato_yellow_leaf_curl_virus       0.57      0.66      0.61       857
      two-spotted_spider_mite       0.58      0.49      0.53       251

                     accuracy                           0.49      2907
                    macro avg       0.53      0.39      0.38      2907
                 weighted avg       0.50      0.49      0.46      2907

[[2

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Logistic on grayscale data

In [55]:
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.4368765049879601

In [56]:
#classification report
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.56      0.50      0.53       349
                 early_blight       0.21      0.15      0.18       182
                      healthy       0.35      0.30      0.32       258
                  late_blight       0.31      0.31      0.31       316
                    leaf_mold       0.41      0.39      0.40       145
           septoria_leaf_spot       0.34      0.35      0.34       262
                  target_spot       0.27      0.23      0.25       229
          tomato_mosaic_virus       0.15      0.19      0.17        58
tomato_yellow_leaf_curl_virus       0.58      0.66      0.62       857
      two-spotted_spider_mite       0.42      0.45      0.43       251

                     accuracy                           0.44      2907
                    macro avg       0.36      0.35      0.36      2907
                 weighted avg       0.43      0.44      0.43      2907

[[1

# SVM(linear) on Grayscale data

In [57]:
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.42346061231510146


In [58]:
#classification report
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.41      0.53      0.46       349
                 early_blight       0.14      0.14      0.14       182
                      healthy       0.43      0.43      0.43       258
                  late_blight       0.29      0.29      0.29       316
                    leaf_mold       0.34      0.35      0.35       145
           septoria_leaf_spot       0.32      0.30      0.31       262
                  target_spot       0.31      0.26      0.28       229
          tomato_mosaic_virus       0.25      0.29      0.27        58
tomato_yellow_leaf_curl_virus       0.61      0.59      0.60       857
      two-spotted_spider_mite       0.49      0.43      0.46       251

                     accuracy                           0.42      2907
                    macro avg       0.36      0.36      0.36      2907
                 weighted avg       0.42      0.42      0.42      2907

[[1

# SVM(poly) on grayscale data

In [59]:
#fitting the model
svm_poly.fit(Ftrain,Ltrain)
y_pred = svm_poly.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.5503955968352253


In [60]:
#classification report
y_predict = svm_poly.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.57      0.56      0.56       349
                 early_blight       0.49      0.12      0.19       182
                      healthy       0.67      0.57      0.61       258
                  late_blight       0.70      0.28      0.40       316
                    leaf_mold       0.69      0.34      0.45       145
           septoria_leaf_spot       0.63      0.42      0.50       262
                  target_spot       0.56      0.36      0.44       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.48      0.90      0.63       857
      two-spotted_spider_mite       0.74      0.56      0.63       251

                     accuracy                           0.55      2907
                    macro avg       0.55      0.41      0.44      2907
                 weighted avg       0.58      0.55      0.52      2907

[[1

# SVM(RBF) on grayscale data

In [61]:
#fitting the model
svm_rbf.fit(Ftrain,Ltrain)
y_pred = svm_rbf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.7010663914688683


In [62]:
#classification report
y_predict = svm_rbf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.74      0.76      0.75       349
                 early_blight       0.53      0.22      0.31       182
                      healthy       0.79      0.88      0.83       258
                  late_blight       0.66      0.66      0.66       316
                    leaf_mold       0.79      0.54      0.64       145
           septoria_leaf_spot       0.64      0.58      0.61       262
                  target_spot       0.67      0.62      0.64       229
          tomato_mosaic_virus       0.78      0.12      0.21        58
tomato_yellow_leaf_curl_virus       0.70      0.87      0.77       857
      two-spotted_spider_mite       0.72      0.70      0.71       251

                     accuracy                           0.70      2907
                    macro avg       0.70      0.59      0.61      2907
                 weighted avg       0.70      0.70      0.69      2907

[[2

# SVM(sigmoid) on grayscale data

In [63]:
#building the model
svm_si.fit(Ftrain,Ltrain)
y_pred = svm_si.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.30787753697970416


In [64]:
#classification report
y_predict = svm_si.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.41      0.45      0.43       349
                 early_blight       0.19      0.05      0.09       182
                      healthy       0.40      0.19      0.26       258
                  late_blight       0.17      0.32      0.22       316
                    leaf_mold       0.59      0.12      0.20       145
           septoria_leaf_spot       0.16      0.40      0.23       262
                  target_spot       0.25      0.09      0.13       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.50      0.43      0.46       857
      two-spotted_spider_mite       0.27      0.26      0.26       251

                     accuracy                           0.31      2907
                    macro avg       0.29      0.23      0.23      2907
                 weighted avg       0.35      0.31      0.30      2907

[[1

# Random Forest on Grayscale

In [65]:
classifier_rf.fit(Ftrain, Ltrain)

RandomForestClassifier(max_depth=5, n_jobs=-1, oob_score=True, random_state=42)

In [66]:
classifier_rf.oob_score_

0.438575361321404

In [67]:
#classification report
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.63      0.46      0.53       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.62      0.34      0.44       258
                  late_blight       0.83      0.18      0.30       316
                    leaf_mold       0.57      0.03      0.05       145
           septoria_leaf_spot       0.58      0.47      0.52       262
                  target_spot       0.85      0.07      0.14       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.38      0.93      0.54       857
      two-spotted_spider_mite       0.61      0.25      0.35       251

                     accuracy                           0.45      2907
                    macro avg       0.51      0.27      0.29      2907
                 weighted avg       0.53      0.45      0.38      2907

[[1

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Gaussian Naive bayes on Grayscale

In [68]:
gnb.fit(Ftrain, Ltrain)

GaussianNB()

In [69]:
#predicting the model
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.3667010663914689

In [70]:
#classification model
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.42      0.73      0.53       349
                 early_blight       0.25      0.31      0.28       182
                      healthy       0.32      0.66      0.43       258
                  late_blight       0.41      0.23      0.29       316
                    leaf_mold       0.45      0.31      0.37       145
           septoria_leaf_spot       0.36      0.50      0.42       262
                  target_spot       0.51      0.34      0.41       229
          tomato_mosaic_virus       0.13      0.60      0.22        58
tomato_yellow_leaf_curl_virus       0.61      0.14      0.23       857
      two-spotted_spider_mite       0.36      0.42      0.39       251

                     accuracy                           0.37      2907
                    macro avg       0.38      0.42      0.36      2907
                 weighted avg       0.45      0.37      0.34      2907

[[2

# Bernouli Naive bayes on grayscale data

In [71]:
bnb.fit(Ftrain, Ltrain)
y_pred = bnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.3202614379084967

In [72]:
#classification report
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.38      0.74      0.50       349
                 early_blight       0.16      0.14      0.15       182
                      healthy       0.28      0.54      0.37       258
                  late_blight       0.19      0.09      0.12       316
                    leaf_mold       0.53      0.28      0.37       145
           septoria_leaf_spot       0.31      0.56      0.40       262
                  target_spot       0.41      0.28      0.34       229
          tomato_mosaic_virus       0.09      0.31      0.14        58
tomato_yellow_leaf_curl_virus       0.53      0.12      0.19       857
      two-spotted_spider_mite       0.35      0.42      0.38       251

                     accuracy                           0.32      2907
                    macro avg       0.32      0.35      0.30      2907
                 weighted avg       0.37      0.32      0.29      2907

[[2

# Decision Tree on Grayscale

In [74]:
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.4293085655314757

In [75]:
#classification report
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.46      0.47      0.46       349
                 early_blight       0.21      0.19      0.20       182
                      healthy       0.58      0.53      0.56       258
                  late_blight       0.40      0.36      0.38       316
                    leaf_mold       0.29      0.35      0.32       145
           septoria_leaf_spot       0.36      0.39      0.37       262
                  target_spot       0.27      0.26      0.26       229
          tomato_mosaic_virus       0.08      0.10      0.09        58
tomato_yellow_leaf_curl_virus       0.55      0.53      0.54       857
      two-spotted_spider_mite       0.39      0.44      0.41       251

                     accuracy                           0.42      2907
                    macro avg       0.36      0.36      0.36      2907
                 weighted avg       0.43      0.42      0.43      2907

[[1

# KNN on Red channel data

In [76]:
#reading numpy array data
red_test = np.load('red_data.npy')
#reading csv file
red_df = pd.read_csv('red_data.csv')

In [77]:
#reading each array and appending in the loaded
loaded = []
for i in red_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
#assigning the features and target
features = loaded
target = red_df['disease']
#splitting the data
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
#scaling the data
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)

In [78]:
#building the model
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.5070519435844513

In [79]:
#classification report
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.44      0.74      0.55       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.38      0.71      0.50       258
                  late_blight       0.48      0.31      0.37       316
                    leaf_mold       0.63      0.44      0.52       145
           septoria_leaf_spot       0.59      0.35      0.44       262
                  target_spot       0.60      0.26      0.36       229
          tomato_mosaic_virus       0.60      0.05      0.10        58
tomato_yellow_leaf_curl_virus       0.57      0.66      0.61       857
      two-spotted_spider_mite       0.54      0.62      0.58       251

                     accuracy                           0.51      2907
                    macro avg       0.48      0.41      0.40      2907
                 weighted avg       0.50      0.51      0.48      2907

[[2

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Logistic on Red channel data

In [80]:
#builiding the model
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.44410044719642244

In [81]:
#classification report
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.55      0.51      0.53       349
                 early_blight       0.20      0.16      0.18       182
                      healthy       0.37      0.33      0.35       258
                  late_blight       0.30      0.29      0.30       316
                    leaf_mold       0.42      0.42      0.42       145
           septoria_leaf_spot       0.33      0.32      0.33       262
                  target_spot       0.32      0.28      0.30       229
          tomato_mosaic_virus       0.16      0.21      0.18        58
tomato_yellow_leaf_curl_virus       0.60      0.66      0.63       857
      two-spotted_spider_mite       0.42      0.46      0.44       251

                     accuracy                           0.44      2907
                    macro avg       0.37      0.37      0.37      2907
                 weighted avg       0.44      0.44      0.44      2907

[[1

# SVM(linear) on Red channel data

In [82]:
#building the model
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.44478844169246645


In [83]:
#classification report
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.43      0.54      0.48       349
                 early_blight       0.21      0.21      0.21       182
                      healthy       0.43      0.42      0.42       258
                  late_blight       0.34      0.37      0.35       316
                    leaf_mold       0.43      0.41      0.42       145
           septoria_leaf_spot       0.34      0.33      0.33       262
                  target_spot       0.39      0.35      0.37       229
          tomato_mosaic_virus       0.23      0.26      0.25        58
tomato_yellow_leaf_curl_virus       0.61      0.59      0.60       857
      two-spotted_spider_mite       0.48      0.37      0.42       251

                     accuracy                           0.44      2907
                    macro avg       0.39      0.39      0.39      2907
                 weighted avg       0.45      0.44      0.45      2907

[[1

# SVM(poly) on Red data

In [84]:
#building the model
svm_poly.fit(Ftrain,Ltrain)
y_pred = svm_poly.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.586171310629515


In [85]:
#classification report
y_predict = svm_poly.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.63      0.58      0.61       349
                 early_blight       0.48      0.08      0.13       182
                      healthy       0.70      0.59      0.64       258
                  late_blight       0.69      0.38      0.49       316
                    leaf_mold       0.70      0.44      0.54       145
           septoria_leaf_spot       0.66      0.47      0.55       262
                  target_spot       0.60      0.46      0.52       229
          tomato_mosaic_virus       0.67      0.03      0.07        58
tomato_yellow_leaf_curl_virus       0.52      0.89      0.65       857
      two-spotted_spider_mite       0.69      0.61      0.64       251

                     accuracy                           0.59      2907
                    macro avg       0.63      0.45      0.48      2907
                 weighted avg       0.61      0.59      0.56      2907

[[2

# SVM(RBF) on Red data

In [86]:
#building the model
svm_rbf.fit(Ftrain,Ltrain)
y_pred = svm_rbf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.7007223942208463


In [87]:
#classification report
y_predict = svm_rbf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.72      0.75      0.74       349
                 early_blight       0.57      0.23      0.33       182
                      healthy       0.80      0.90      0.85       258
                  late_blight       0.65      0.67      0.66       316
                    leaf_mold       0.75      0.54      0.63       145
           septoria_leaf_spot       0.60      0.54      0.57       262
                  target_spot       0.65      0.61      0.63       229
          tomato_mosaic_virus       0.54      0.12      0.20        58
tomato_yellow_leaf_curl_virus       0.72      0.86      0.79       857
      two-spotted_spider_mite       0.71      0.72      0.71       251

                     accuracy                           0.70      2907
                    macro avg       0.67      0.60      0.61      2907
                 weighted avg       0.69      0.70      0.69      2907

[[2

# SVM(sigmoid) on Red data

In [88]:
#building the model
svm_si.fit(Ftrain,Ltrain)
y_pred = svm_si.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.3457172342621259


In [89]:
#classification report
y_predict = svm_si.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.41      0.52      0.46       349
                 early_blight       0.07      0.02      0.03       182
                      healthy       0.44      0.25      0.32       258
                  late_blight       0.19      0.29      0.23       316
                    leaf_mold       0.58      0.12      0.20       145
           septoria_leaf_spot       0.18      0.38      0.24       262
                  target_spot       0.26      0.19      0.22       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.53      0.51      0.52       857
      two-spotted_spider_mite       0.36      0.25      0.30       251

                     accuracy                           0.35      2907
                    macro avg       0.30      0.25      0.25      2907
                 weighted avg       0.37      0.35      0.34      2907

[[1

# Random forest on Red channel data

In [90]:
#building the model
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

0.4755677907777013

In [91]:
#classification report
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.66      0.48      0.55       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.65      0.36      0.46       258
                  late_blight       0.65      0.19      0.29       316
                    leaf_mold       0.80      0.08      0.15       145
           septoria_leaf_spot       0.54      0.48      0.51       262
                  target_spot       0.79      0.17      0.27       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.41      0.90      0.56       857
      two-spotted_spider_mite       0.54      0.51      0.52       251

                     accuracy                           0.48      2907
                    macro avg       0.50      0.32      0.33      2907
                 weighted avg       0.53      0.48      0.42      2907

[[1

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Gaussian Naive Bayes on Red channel data

In [92]:
#building the model
gnb.fit(Ftrain, Ltrain)

GaussianNB()

In [93]:
#predicting the model
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.38390092879256965

In [94]:
#classification report
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.46      0.73      0.56       349
                 early_blight       0.22      0.24      0.23       182
                      healthy       0.35      0.72      0.47       258
                  late_blight       0.45      0.24      0.32       316
                    leaf_mold       0.38      0.31      0.34       145
           septoria_leaf_spot       0.35      0.46      0.40       262
                  target_spot       0.50      0.40      0.45       229
          tomato_mosaic_virus       0.14      0.64      0.24        58
tomato_yellow_leaf_curl_virus       0.64      0.15      0.24       857
      two-spotted_spider_mite       0.39      0.54      0.46       251

                     accuracy                           0.38      2907
                    macro avg       0.39      0.44      0.37      2907
                 weighted avg       0.46      0.38      0.36      2907

[[2

# Bernouli Naive bayes on Red data

In [95]:
#fitting the model
bnb.fit(Ftrain, Ltrain)
y_pred = bnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.3347093223254214

In [96]:
#classification report
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.41      0.74      0.53       349
                 early_blight       0.16      0.12      0.14       182
                      healthy       0.29      0.60      0.39       258
                  late_blight       0.21      0.10      0.14       316
                    leaf_mold       0.35      0.16      0.22       145
           septoria_leaf_spot       0.31      0.49      0.38       262
                  target_spot       0.43      0.36      0.39       229
          tomato_mosaic_virus       0.12      0.36      0.18        58
tomato_yellow_leaf_curl_virus       0.59      0.15      0.24       857
      two-spotted_spider_mite       0.32      0.50      0.39       251

                     accuracy                           0.33      2907
                    macro avg       0.32      0.36      0.30      2907
                 weighted avg       0.39      0.33      0.30      2907

[[2

# Decision Tree on Red channel

In [97]:
#building the model
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.43584451324389406

In [98]:
#classification report
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.48      0.48      0.48       349
                 early_blight       0.17      0.15      0.16       182
                      healthy       0.56      0.55      0.56       258
                  late_blight       0.39      0.34      0.36       316
                    leaf_mold       0.26      0.26      0.26       145
           septoria_leaf_spot       0.34      0.45      0.39       262
                  target_spot       0.28      0.26      0.27       229
          tomato_mosaic_virus       0.09      0.10      0.10        58
tomato_yellow_leaf_curl_virus       0.56      0.55      0.56       857
      two-spotted_spider_mite       0.41      0.41      0.41       251

                     accuracy                           0.43      2907
                    macro avg       0.35      0.36      0.35      2907
                 weighted avg       0.43      0.43      0.43      2907

[[1

# KNN on green channel data

In [99]:
#reading numpy array data
green_test = np.load('green_data.npy')
#reading csv file
green_df = pd.read_csv('green_data.csv')
#reading each array and appending to loaded
loaded = []
for i in green_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
#assigning the features and target
features = loaded
target = green_df['disease']
#splitting the data to test and train
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
#scaling the data
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)
#building the model
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.48297213622291024

In [100]:
#classification report
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.35      0.76      0.48       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.38      0.57      0.46       258
                  late_blight       0.35      0.30      0.32       316
                    leaf_mold       0.62      0.44      0.52       145
           septoria_leaf_spot       0.58      0.41      0.48       262
                  target_spot       0.71      0.13      0.22       229
          tomato_mosaic_virus       1.00      0.02      0.03        58
tomato_yellow_leaf_curl_virus       0.61      0.68      0.64       857
      two-spotted_spider_mite       0.56      0.43      0.49       251

                     accuracy                           0.48      2907
                    macro avg       0.52      0.37      0.36      2907
                 weighted avg       0.50      0.48      0.45      2907

[[2

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Logistic on Green channel data

In [101]:
#building the model
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.4454764361885105

In [102]:
#classification report
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.53      0.50      0.51       349
                 early_blight       0.21      0.17      0.19       182
                      healthy       0.39      0.36      0.38       258
                  late_blight       0.34      0.31      0.33       316
                    leaf_mold       0.36      0.32      0.34       145
           septoria_leaf_spot       0.31      0.31      0.31       262
                  target_spot       0.26      0.22      0.24       229
          tomato_mosaic_virus       0.17      0.21      0.19        58
tomato_yellow_leaf_curl_virus       0.61      0.69      0.65       857
      two-spotted_spider_mite       0.40      0.45      0.43       251

                     accuracy                           0.45      2907
                    macro avg       0.36      0.35      0.36      2907
                 weighted avg       0.43      0.45      0.44      2907

[[1

# SVM(linear) on green channel data

In [103]:
#building the model
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.4286205710354317


In [104]:
#classification report
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.39      0.49      0.43       349
                 early_blight       0.15      0.16      0.15       182
                      healthy       0.46      0.46      0.46       258
                  late_blight       0.33      0.32      0.32       316
                    leaf_mold       0.31      0.32      0.31       145
           septoria_leaf_spot       0.32      0.27      0.29       262
                  target_spot       0.33      0.29      0.31       229
          tomato_mosaic_virus       0.19      0.22      0.21        58
tomato_yellow_leaf_curl_virus       0.63      0.61      0.62       857
      two-spotted_spider_mite       0.46      0.42      0.44       251

                     accuracy                           0.43      2907
                    macro avg       0.36      0.36      0.36      2907
                 weighted avg       0.43      0.43      0.43      2907

[[1

# SVM(poly) on Green data

In [105]:
#building the model
svm_poly.fit(Ftrain,Ltrain)
y_pred = svm_poly.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.5311317509459924


In [106]:
#classification report
y_predict = svm_poly.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.56      0.54      0.55       349
                 early_blight       0.50      0.14      0.22       182
                      healthy       0.65      0.54      0.59       258
                  late_blight       0.69      0.20      0.31       316
                    leaf_mold       0.64      0.26      0.37       145
           septoria_leaf_spot       0.61      0.39      0.48       262
                  target_spot       0.61      0.32      0.42       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.46      0.91      0.61       857
      two-spotted_spider_mite       0.76      0.54      0.63       251

                     accuracy                           0.53      2907
                    macro avg       0.55      0.38      0.42      2907
                 weighted avg       0.57      0.53      0.50      2907

[[1

# SVM(RBF) on green data

In [107]:
#building the model
svm_rbf.fit(Ftrain,Ltrain)
y_pred = svm_rbf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.7103543171654627


In [108]:
#classification report
y_predict = svm_rbf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.75      0.76      0.76       349
                 early_blight       0.54      0.25      0.34       182
                      healthy       0.78      0.87      0.82       258
                  late_blight       0.67      0.66      0.67       316
                    leaf_mold       0.70      0.57      0.63       145
           septoria_leaf_spot       0.64      0.60      0.62       262
                  target_spot       0.66      0.60      0.63       229
          tomato_mosaic_virus       0.82      0.16      0.26        58
tomato_yellow_leaf_curl_virus       0.72      0.88      0.79       857
      two-spotted_spider_mite       0.76      0.71      0.73       251

                     accuracy                           0.71      2907
                    macro avg       0.70      0.61      0.63      2907
                 weighted avg       0.70      0.71      0.70      2907

[[2

# SVM(sigmoid) on green data

In [109]:
#building the model
svm_si.fit(Ftrain,Ltrain)
y_pred = svm_si.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.29239766081871343


In [110]:
#classification report
y_predict = svm_si.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.36      0.42      0.39       349
                 early_blight       0.15      0.05      0.08       182
                      healthy       0.33      0.39      0.36       258
                  late_blight       0.16      0.24      0.19       316
                    leaf_mold       0.52      0.08      0.13       145
           septoria_leaf_spot       0.13      0.27      0.17       262
                  target_spot       0.28      0.07      0.11       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.44      0.44      0.44       857
      two-spotted_spider_mite       0.25      0.17      0.20       251

                     accuracy                           0.29      2907
                    macro avg       0.26      0.21      0.21      2907
                 weighted avg       0.31      0.29      0.28      2907

[[1

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Random Forest on green channel data

In [111]:
#building the model
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

0.4253269098417068

In [112]:
#classification report
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.56      0.46      0.51       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.61      0.42      0.50       258
                  late_blight       0.69      0.24      0.36       316
                    leaf_mold       0.00      0.00      0.00       145
           septoria_leaf_spot       0.60      0.43      0.50       262
                  target_spot       0.00      0.00      0.00       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.37      0.91      0.53       857
      two-spotted_spider_mite       0.64      0.06      0.12       251

                     accuracy                           0.43      2907
                    macro avg       0.35      0.25      0.25      2907
                 weighted avg       0.42      0.43      0.35      2907

[[1

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Gaussian Naive bayes on Green channel data

In [113]:
#building the model
gnb.fit(Ftrain, Ltrain)
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.35053319573443414

In [114]:
#classification report
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.40      0.72      0.51       349
                 early_blight       0.26      0.35      0.29       182
                      healthy       0.35      0.63      0.45       258
                  late_blight       0.41      0.22      0.29       316
                    leaf_mold       0.41      0.29      0.34       145
           septoria_leaf_spot       0.30      0.49      0.37       262
                  target_spot       0.49      0.33      0.40       229
          tomato_mosaic_virus       0.12      0.52      0.19        58
tomato_yellow_leaf_curl_virus       0.56      0.11      0.19       857
      two-spotted_spider_mite       0.35      0.38      0.37       251

                     accuracy                           0.35      2907
                    macro avg       0.36      0.40      0.34      2907
                 weighted avg       0.42      0.35      0.32      2907

[[2

# Bernouli Naive bayes on green data

In [115]:
#building the model
bnb.fit(Ftrain, Ltrain)
y_pred = bnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.2844857241142071

In [116]:
#classification report
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.37      0.72      0.49       349
                 early_blight       0.17      0.18      0.17       182
                      healthy       0.27      0.52      0.36       258
                  late_blight       0.25      0.13      0.17       316
                    leaf_mold       0.50      0.22      0.31       145
           septoria_leaf_spot       0.28      0.57      0.37       262
                  target_spot       0.40      0.20      0.27       229
          tomato_mosaic_virus       0.07      0.31      0.11        58
tomato_yellow_leaf_curl_virus       0.30      0.04      0.07       857
      two-spotted_spider_mite       0.32      0.35      0.33       251

                     accuracy                           0.28      2907
                    macro avg       0.29      0.32      0.27      2907
                 weighted avg       0.30      0.28      0.24      2907

[[2

# Decision Tree on Green channel

In [117]:
#building the model
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.413140694874441

In [118]:
#classification report
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.41      0.44      0.43       349
                 early_blight       0.13      0.13      0.13       182
                      healthy       0.56      0.56      0.56       258
                  late_blight       0.39      0.32      0.35       316
                    leaf_mold       0.21      0.26      0.23       145
           septoria_leaf_spot       0.36      0.43      0.39       262
                  target_spot       0.26      0.25      0.26       229
          tomato_mosaic_virus       0.12      0.10      0.11        58
tomato_yellow_leaf_curl_virus       0.58      0.53      0.55       857
      two-spotted_spider_mite       0.34      0.41      0.37       251

                     accuracy                           0.41      2907
                    macro avg       0.34      0.34      0.34      2907
                 weighted avg       0.42      0.41      0.41      2907

[[1

# KNN on blue channel data

In [119]:
#reading numpy array data
blue_test = np.load('blue_data.npy')
#reading csv file
blue_df = pd.read_csv('blue_data.csv')
#loading the data to loaded array
loaded = []
for i in gray_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
#assigning features and target
features = loaded
target = blue_df['disease']
#splitting the data
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
#scaling the data
Ftrain = scaler.fit_transform(Ftrain)
Ftest = scaler.fit_transform(Ftest)
#fitting the model
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.4864121087031304

In [120]:
#classification report
y_predict = kmodel.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.38      0.77      0.50       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.38      0.60      0.46       258
                  late_blight       0.37      0.29      0.32       316
                    leaf_mold       0.74      0.46      0.56       145
           septoria_leaf_spot       0.56      0.39      0.46       262
                  target_spot       0.70      0.17      0.27       229
          tomato_mosaic_virus       1.00      0.03      0.07        58
tomato_yellow_leaf_curl_virus       0.57      0.66      0.61       857
      two-spotted_spider_mite       0.58      0.49      0.53       251

                     accuracy                           0.49      2907
                    macro avg       0.53      0.39      0.38      2907
                 weighted avg       0.50      0.49      0.46      2907

[[2

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# logistic on blue channel data

In [121]:
#building a model
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.4368765049879601

In [122]:
#classification report
y_predict = model.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.56      0.50      0.53       349
                 early_blight       0.21      0.15      0.18       182
                      healthy       0.35      0.30      0.32       258
                  late_blight       0.31      0.31      0.31       316
                    leaf_mold       0.41      0.39      0.40       145
           septoria_leaf_spot       0.34      0.35      0.34       262
                  target_spot       0.27      0.23      0.25       229
          tomato_mosaic_virus       0.15      0.19      0.17        58
tomato_yellow_leaf_curl_virus       0.58      0.66      0.62       857
      two-spotted_spider_mite       0.42      0.45      0.43       251

                     accuracy                           0.44      2907
                    macro avg       0.36      0.35      0.36      2907
                 weighted avg       0.43      0.44      0.43      2907

[[1

# SVM(linear) on blue channel data

In [123]:
#fitting the model
clf.fit(Ftrain,Ltrain)
#predicting the model
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.42346061231510146


In [124]:
#classification report
y_predict = clf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.41      0.53      0.46       349
                 early_blight       0.14      0.14      0.14       182
                      healthy       0.43      0.43      0.43       258
                  late_blight       0.29      0.29      0.29       316
                    leaf_mold       0.34      0.35      0.35       145
           septoria_leaf_spot       0.32      0.30      0.31       262
                  target_spot       0.31      0.26      0.28       229
          tomato_mosaic_virus       0.25      0.29      0.27        58
tomato_yellow_leaf_curl_virus       0.61      0.59      0.60       857
      two-spotted_spider_mite       0.49      0.43      0.46       251

                     accuracy                           0.42      2907
                    macro avg       0.36      0.36      0.36      2907
                 weighted avg       0.42      0.42      0.42      2907

[[1

# SVM(poly) on blue data

In [125]:
#fitting the model
svm_poly.fit(Ftrain,Ltrain)
y_pred = svm_poly.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.5503955968352253


In [126]:
#classification report
y_predict = svm_poly.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.57      0.56      0.56       349
                 early_blight       0.49      0.12      0.19       182
                      healthy       0.67      0.57      0.61       258
                  late_blight       0.70      0.28      0.40       316
                    leaf_mold       0.69      0.34      0.45       145
           septoria_leaf_spot       0.63      0.42      0.50       262
                  target_spot       0.56      0.36      0.44       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.48      0.90      0.63       857
      two-spotted_spider_mite       0.74      0.56      0.63       251

                     accuracy                           0.55      2907
                    macro avg       0.55      0.41      0.44      2907
                 weighted avg       0.58      0.55      0.52      2907

[[1

# SVM(RBF) on blue data

In [127]:
#building the model
svm_rbf.fit(Ftrain,Ltrain)
y_pred = svm_rbf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.7010663914688683


In [128]:
#classification report
y_predict = svm_rbf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.74      0.76      0.75       349
                 early_blight       0.53      0.22      0.31       182
                      healthy       0.79      0.88      0.83       258
                  late_blight       0.66      0.66      0.66       316
                    leaf_mold       0.79      0.54      0.64       145
           septoria_leaf_spot       0.64      0.58      0.61       262
                  target_spot       0.67      0.62      0.64       229
          tomato_mosaic_virus       0.78      0.12      0.21        58
tomato_yellow_leaf_curl_virus       0.70      0.87      0.77       857
      two-spotted_spider_mite       0.72      0.70      0.71       251

                     accuracy                           0.70      2907
                    macro avg       0.70      0.59      0.61      2907
                 weighted avg       0.70      0.70      0.69      2907

[[2

# SVM(sigmoid) on blue data

In [129]:
#building the model
svm_si.fit(Ftrain,Ltrain)
y_pred = svm_si.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.30787753697970416


In [130]:
#classification report
y_predict = svm_si.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.41      0.45      0.43       349
                 early_blight       0.19      0.05      0.09       182
                      healthy       0.40      0.19      0.26       258
                  late_blight       0.17      0.32      0.22       316
                    leaf_mold       0.59      0.12      0.20       145
           septoria_leaf_spot       0.16      0.40      0.23       262
                  target_spot       0.25      0.09      0.13       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.50      0.43      0.46       857
      two-spotted_spider_mite       0.27      0.26      0.26       251

                     accuracy                           0.31      2907
                    macro avg       0.29      0.23      0.23      2907
                 weighted avg       0.35      0.31      0.30      2907

[[1

# Random Forest on blue channel data

In [131]:
#building the model
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

0.438575361321404

In [132]:
#classification report
y_predict = classifier_rf.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.63      0.46      0.53       349
                 early_blight       0.00      0.00      0.00       182
                      healthy       0.62      0.34      0.44       258
                  late_blight       0.83      0.18      0.30       316
                    leaf_mold       0.57      0.03      0.05       145
           septoria_leaf_spot       0.58      0.47      0.52       262
                  target_spot       0.85      0.07      0.14       229
          tomato_mosaic_virus       0.00      0.00      0.00        58
tomato_yellow_leaf_curl_virus       0.38      0.93      0.54       857
      two-spotted_spider_mite       0.61      0.25      0.35       251

                     accuracy                           0.45      2907
                    macro avg       0.51      0.27      0.29      2907
                 weighted avg       0.53      0.45      0.38      2907

[[1

C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ravic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Gaussian Naive bayes on blue channel data

In [133]:
#building model
gnb.fit(Ftrain, Ltrain)
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.3667010663914689

In [134]:
#classification report
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.42      0.73      0.53       349
                 early_blight       0.25      0.31      0.28       182
                      healthy       0.32      0.66      0.43       258
                  late_blight       0.41      0.23      0.29       316
                    leaf_mold       0.45      0.31      0.37       145
           septoria_leaf_spot       0.36      0.50      0.42       262
                  target_spot       0.51      0.34      0.41       229
          tomato_mosaic_virus       0.13      0.60      0.22        58
tomato_yellow_leaf_curl_virus       0.61      0.14      0.23       857
      two-spotted_spider_mite       0.36      0.42      0.39       251

                     accuracy                           0.37      2907
                    macro avg       0.38      0.42      0.36      2907
                 weighted avg       0.45      0.37      0.34      2907

[[2

# Bernouli Naive bayes on blue data

In [135]:
#building the model
bnb.fit(Ftrain, Ltrain)
y_pred = bnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

0.3202614379084967

In [136]:
#classification report
class_names = target.unique()
print(classification_report(Ltest, y_pred, target_names=class_names))
print(confusion_matrix(Ltest, y_pred))

                               precision    recall  f1-score   support

               bacterial_spot       0.38      0.74      0.50       349
                 early_blight       0.16      0.14      0.15       182
                      healthy       0.28      0.54      0.37       258
                  late_blight       0.19      0.09      0.12       316
                    leaf_mold       0.53      0.28      0.37       145
           septoria_leaf_spot       0.31      0.56      0.40       262
                  target_spot       0.41      0.28      0.34       229
          tomato_mosaic_virus       0.09      0.31      0.14        58
tomato_yellow_leaf_curl_virus       0.53      0.12      0.19       857
      two-spotted_spider_mite       0.35      0.42      0.38       251

                     accuracy                           0.32      2907
                    macro avg       0.32      0.35      0.30      2907
                 weighted avg       0.37      0.32      0.29      2907

[[2

# Decision Tree on Blue channel

In [137]:
#building the model
classifier_tree.fit(Ftrain, Ltrain)
classifier_tree.score(Ftest,Ltest)

0.4262125902992776

In [138]:
#classification report
y_predict = classifier_tree.fit(Ftrain, Ltrain).predict(Ftest)
class_names = target.unique()
print(classification_report(Ltest, y_predict, target_names=class_names))
print(confusion_matrix(Ltest, y_predict))

                               precision    recall  f1-score   support

               bacterial_spot       0.44      0.45      0.45       349
                 early_blight       0.19      0.18      0.18       182
                      healthy       0.60      0.55      0.58       258
                  late_blight       0.43      0.37      0.40       316
                    leaf_mold       0.27      0.33      0.30       145
           septoria_leaf_spot       0.40      0.45      0.42       262
                  target_spot       0.25      0.24      0.24       229
          tomato_mosaic_virus       0.06      0.07      0.06        58
tomato_yellow_leaf_curl_virus       0.53      0.52      0.53       857
      two-spotted_spider_mite       0.39      0.42      0.40       251

                     accuracy                           0.42      2907
                    macro avg       0.36      0.36      0.36      2907
                 weighted avg       0.42      0.42      0.42      2907

[[1